In [3]:
#1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set(style="whitegrid")


fname = "BIKE DETAILS.csv"
df = pd.read_csv(fname)


out = "eda_outputs"
os.makedirs(out, exist_ok=True)


print("Q1: shape:", df.shape)
print("Q1: columns:", list(df.columns))
display_head = df.head(10)
print(display_head)
display_head.to_csv(os.path.join(out, "q1_first10.csv"), index=False)

Q1: shape: (1061, 7)
Q1: columns: ['name', 'selling_price', 'year', 'seller_type', 'owner', 'km_driven', 'ex_showroom_price']
                                   name  selling_price  year seller_type  \
0             Royal Enfield Classic 350         175000  2019  Individual   
1                             Honda Dio          45000  2017  Individual   
2   Royal Enfield Classic Gunmetal Grey         150000  2018  Individual   
3     Yamaha Fazer FI V 2.0 [2016-2018]          65000  2015  Individual   
4                 Yamaha SZ [2013-2014]          20000  2011  Individual   
5                      Honda CB Twister          18000  2010  Individual   
6                  Honda CB Hornet 160R          78500  2018  Individual   
7  Royal Enfield Bullet 350 [2007-2011]         180000  2008  Individual   
8                Hero Honda CBZ extreme          30000  2010  Individual   
9                    Bajaj Discover 125          50000  2016  Individual   

       owner  km_driven  ex_showroom_

In [6]:
#2
print("\nQ2: Missing values per column:")
missing = df.isna().sum()
print(missing)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()


print("\nMissing % by column:")
print((df.isna().mean()*100).round(2))

df_clean = df.copy()
for c in num_cols:
    if df_clean[c].isna().sum() > 0:
        med = df_clean[c].median()
        df_clean[c].fillna(med, inplace=True)
for c in cat_cols:
    if df_clean[c].isna().sum() > 0:
        mode = df_clean[c].mode()
        if len(mode) > 0:
            df_clean[c].fillna(mode[0], inplace=True)
        else:
            df_clean[c].fillna("Unknown", inplace=True)


missing.to_csv(os.path.join(out, "q2_missing_counts.csv"))


Q2: Missing values per column:
name                   0
selling_price          0
year                   0
seller_type            0
owner                  0
km_driven              0
ex_showroom_price    435
dtype: int64

Missing % by column:
name                  0.0
selling_price         0.0
year                  0.0
seller_type           0.0
owner                 0.0
km_driven             0.0
ex_showroom_price    41.0
dtype: float64


/tmp/ipython-input-1658941745.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[c].fillna(med, inplace=True)


In [7]:
#3
plt.figure(figsize=(8,5))
sns.histplot(df_clean['selling_price'], kde=False, bins=30)
plt.title("Distribution of Selling Price")
plt.xlabel("Selling Price")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(os.path.join(out, "q3_selling_price_hist.png"), dpi=150)
plt.close()

print("\nQ3: selling_price summary statistics:")
print(df_clean['selling_price'].describe())


Q3: selling_price summary statistics:
count      1061.000000
mean      59638.151744
std       56304.291973
min        5000.000000
25%       28000.000000
50%       45000.000000
75%       70000.000000
max      760000.000000
Name: selling_price, dtype: float64


In [8]:
#4
if 'seller_type' in df_clean.columns:
    avg_by_seller = df_clean.groupby('seller_type')['selling_price'].mean().sort_values()
    print("\nQ4: average selling price by seller_type:")
    print(avg_by_seller)
    plt.figure(figsize=(7,4))
    sns.barplot(x=avg_by_seller.index, y=avg_by_seller.values)
    plt.ylabel("Average Selling Price")
    plt.xlabel("Seller Type")
    plt.title("Avg Selling Price by Seller Type")
    plt.tight_layout()
    plt.savefig(os.path.join(out, "q4_avg_price_by_seller_type.png"), dpi=150)
    plt.close()
else:
    print("Q4: Column 'seller_type' not found.")


Q4: average selling price by seller_type:
seller_type
Dealer        46666.666667
Individual    59711.923223
Name: selling_price, dtype: float64


In [9]:
#5
if 'owner' in df_clean.columns and 'km_driven' in df_clean.columns:
    avg_km_by_owner = df_clean.groupby('owner')['km_driven'].mean().sort_values()
    print("\nQ5: avg km_driven by owner:")
    print(avg_km_by_owner)
    plt.figure(figsize=(8,4))
    sns.barplot(x=avg_km_by_owner.index, y=avg_km_by_owner.values)
    plt.ylabel("Average km_driven")
    plt.xlabel("Ownership Type")
    plt.title("Avg km_driven by Ownership Type")
    plt.tight_layout()
    plt.savefig(os.path.join(out, "q5_avg_km_by_owner.png"), dpi=150)
    plt.close()
else:
    print("Q5: Columns 'owner' and/or 'km_driven' not found.")


Q5: avg km_driven by owner:
owner
1st owner     32816.583333
3rd owner     33292.181818
2nd owner     39288.991870
4th owner    311500.000000
Name: km_driven, dtype: float64


In [10]:
#6
if 'km_driven' in df_clean.columns:
    before_stats = df_clean['km_driven'].describe()
    Q1 = df_clean['km_driven'].quantile(0.25)
    Q3 = df_clean['km_driven'].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    mask = (df_clean['km_driven'] >= lower) & (df_clean['km_driven'] <= upper)
    df_no_out = df_clean[mask].copy()
    after_stats = df_no_out['km_driven'].describe()
    print("\nQ6: km_driven before:\n", before_stats)
    print("\nQ6: km_driven after removing outliers:\n", after_stats)

    pd.DataFrame({'before': before_stats, 'after': after_stats}).to_csv(os.path.join(out, "q6_km_stats_before_after.csv"))
else:
    print("Q6: 'km_driven' not found.")


Q6: km_driven before:
 count      1061.000000
mean      34359.833176
std       51623.152702
min         350.000000
25%       13500.000000
50%       25000.000000
75%       43000.000000
max      880000.000000
Name: km_driven, dtype: float64

Q6: km_driven after removing outliers:
 count     1022.000000
mean     28203.415851
std      19552.083583
min        350.000000
25%      13000.000000
50%      24000.000000
75%      40000.000000
max      86000.000000
Name: km_driven, dtype: float64


In [13]:
#7
if 'year' in df_clean.columns and 'selling_price' in df_clean.columns:
    plt.figure(figsize=(8,5))
    sns.scatterplot(x='year', y='selling_price', data=df_clean, alpha=0.5)
    plt.title("Year vs Selling Price")
    plt.xlabel("Year")
    plt.ylabel("Selling Price")
    plt.tight_layout()
    plt.savefig(os.path.join(out, "q7_year_vs_price_scatter.png"), dpi=150)
    plt.close()
else:
    print("Q7: 'year' and/or 'selling_price' not found.")

In [14]:
#8
if 'seller_type' in df_clean.columns:
    df_ohe = pd.get_dummies(df_clean, columns=['seller_type'], drop_first=False)
    print("\nQ8: first 5 rows after one-hot encoding seller_type:")
    print(df_ohe.head(5))
    df_ohe.head(5).to_csv(os.path.join(out, "q8_ohe_first5.csv"), index=False)
else:
    print("Q8: 'seller_type' not found. Skipping OHE.")


Q8: first 5 rows after one-hot encoding seller_type:
                                  name  selling_price  year      owner  \
0            Royal Enfield Classic 350         175000  2019  1st owner   
1                            Honda Dio          45000  2017  1st owner   
2  Royal Enfield Classic Gunmetal Grey         150000  2018  1st owner   
3    Yamaha Fazer FI V 2.0 [2016-2018]          65000  2015  1st owner   
4                Yamaha SZ [2013-2014]          20000  2011  2nd owner   

   km_driven  ex_showroom_price  seller_type_Dealer  seller_type_Individual  
0        350            72752.5               False                    True  
1       5650            72752.5               False                    True  
2      12000           148114.0               False                    True  
3      23000            89643.0               False                    True  
4      21000            72752.5               False                    True  


In [15]:
#9
num_df = df_clean.select_dtypes(include=[np.number]).copy()
corr = num_df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=False)
plt.title("Correlation matrix (numeric columns)")
plt.tight_layout()
plt.savefig(os.path.join(out, "q9_correlation_heatmap.png"), dpi=150)
plt.close()
print("\nQ9: Top correlations (absolute) — head of pairs:")
corr_unstack = corr.abs().unstack().sort_values(ascending=False)

corr_unstack = corr_unstack[corr_unstack < 0.9999]
top_pairs = corr_unstack.drop_duplicates().head(10)
print(top_pairs)


Q9: Top correlations (absolute) — head of pairs:
selling_price  ex_showroom_price    0.744075
year           selling_price        0.402188
               km_driven            0.288675
km_driven      selling_price        0.212937
year           ex_showroom_price    0.100066
km_driven      ex_showroom_price    0.086791
dtype: float64


In [16]:
#10
print("\nQ10: Summary of findings (brief):")
print("""
- Selling price tends to be strongly related to the bike's year (newer bikes command higher prices).
- Variables like 'km_driven' and 'owner' also influence price (higher km usually lowers price).
- We cleaned missing values by filling numeric NaNs with medians and categorical with modes.
- We removed km_driven outliers using the IQR method to avoid skew in km-related analyses.
- One-hot encoding was applied to seller_type for modeling readiness.
""")


df_clean.to_csv(os.path.join(out, "df_clean.csv"), index=False)
if 'df_no_out' in locals():
    df_no_out.to_csv(os.path.join(out, "df_no_outliers.csv"), index=False)

print(f"\nAll outputs saved in folder: {out}")



Q10: Summary of findings (brief):

- Selling price tends to be strongly related to the bike's year (newer bikes command higher prices).
- Variables like 'km_driven' and 'owner' also influence price (higher km usually lowers price).
- We cleaned missing values by filling numeric NaNs with medians and categorical with modes.
- We removed km_driven outliers using the IQR method to avoid skew in km-related analyses.
- One-hot encoding was applied to seller_type for modeling readiness.


All outputs saved in folder: eda_outputs
